# Topic Modelling using LDA

## Notebook Overview
In this notebook, we will explore topic modelling using Latent Dirichlet Allocation (LDA) on a corpus of Geological Survey of Western Australia reports.

⚠️This notebook takes approximately 50 minutes to execute all cells. The visualisations of the topic models is the culprit.

## Table of Contents
1. [Loading GSWA Corpus](#load_corpus)
2. [Corpus Pre-processing](#preprocessing)
3. [Training LDA Model](#training)
4. [Interactive Visualisation](#visualisation)
5. [Additional Content: LDA Model Hyperparameter Tuning](#tuning)

## Notebook Setup

In [1]:
!pip install pyLDAvis gensim==4.2.0 spacy matplotlib numpy pandas nltk

You should consider upgrading via the 'c:\users\tyler\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


We need to load our pre-trained model from SpaCy before we can use it.

In [2]:
!pip install -U spacy download en_core_web_sm

You should consider upgrading via the 'c:\users\tyler\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Lets load in the package [nb_black](https://github.com/dnanhkhoa/nb_black) to auto-format our code as we go. Formatting code is useful as it easier to collaborate and interpret what we write.

In [3]:
# Package for ensuring code we write is formatted nicely (see: https://github.com/dnanhkhoa/nb_black)
%load_ext nb_black

<IPython.core.display.Javascript object>

### Import Packages
Standard packages
- [re](https://docs.python.org/3/library/re.html) - library that we will use for performing regular expressions

In [4]:
import re
from pprint import pprint
import warnings
import zipfile
import json
from typing import List

<IPython.core.display.Javascript object>

Third party packages

- [numpy](https://numpy.org/) - library that we will use for helping visualise results
- [pandas](https://pandas.pydata.org/) - library that we will use to help handling our corpus
- [gensim](https://radimrehurek.com/gensim/) - library that we will use to perform topic modelling
- [spacy](https://spacy.io/) - library that we will use to perform lemmatization
- [nltk](https://www.nltk.org/) - library that we will use to pre-process our corpus
- [pyLDAvis](https://pyldavis.readthedocs.io/en/latest/readme.html) - library that we will use to interactively visualise results

In [5]:
import nltk
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

c:\Users\Tyler\AppData\Local\Programs\Python\Python38\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


<IPython.core.display.Javascript object>

### Set-up notebook environment and helper functions

In [6]:
# Download NLTK stopwords
nltk.download('stopwords', quiet=True)

True

<IPython.core.display.Javascript object>

In [7]:
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)

<IPython.core.display.Javascript object>

## Import the GSWA dataset <a name="load_corpus"></a>

The 20-Newsgroups dataset contains about 6k data posts from 1,135 GSWA reports. 

The dataset can be imported using `pandas.read_json` and we can see the dataset has 3 columns. The `target-name` is the category or topics that the news is manually assigned to. 

We choose to use this dataset because it has pre-assigned categories according to topics, so our clustering results can be readily compared. In real world data, this is not always easily available. 

In [8]:
corpus_path = r"../data/wamex_xml.zip"

<IPython.core.display.Javascript object>

In [9]:
corpus = list()
with zipfile.ZipFile(corpus_path, "r") as z:
    for filename in z.namelist():
        with z.open(filename) as f:
            # load the json file
            # The resulting `content` is a list
            content = json.loads(f.read()) 
            # Convert content to a string   
            content = "".join(content)
            # Add to the data list
            corpus.append(content)

<IPython.core.display.Javascript object>

In [10]:
pprint(corpus[:1])

['Combined Reporting Application Pending Status: Sheet 1:250 000: Kalgoorlie '
 '(SH 51-09) Sheet 1:100 000: Kalgoorlie (3136) Project Operator: Cazaly '
 'Resources Limited Author: M Watts Date: September 2006 Distribution: '
 '1.Department of Industry and Resources 2.Cazaly Resources Limited Cazaly '
 'Resources Limited September 2006 2 Figure 2: Castle Hill Project Tenement '
 'Location Plan, 1: 75 000 scale 6 Figure 3: Regional Geology with Exploration '
 'Index Map, 1:75 000 scale 8Cazaly Resources Limited September 2006 3 1.0 '
 'SUMMARY The Combined Mineral Exploration Report on the Castle Hill Project '
 'details exploration activities undertaken by Cazaly Resources Limited during '
 'the reporting period from 7 July 2005 to 6 July 2006.The reporting group '
 'known as Castle Hill comprises nine granted prospecting licences.The project '
 'area covers some 1250 hectares.Cazaly is the registered holder of the Castle '
 'Hill Project tenements.Cazalys exploration activities durin

<IPython.core.display.Javascript object>

## Pre-processing <a name="preprocessing"></a>

Before performing topic modelling, we need to pre-process our corpus. Here we will remove unnecessary information using regular expressions via `re`.

In [11]:
# Create set of English stopwords and include domain-specific words 
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

<IPython.core.display.Javascript object>

#Initialize spacy 'en' model, keeping only tagger component (for efficiency)

In [ ]:
from spacy.cli import download
download("en_core_web_sm")

In [12]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

<IPython.core.display.Javascript object>

In [13]:
def lemmatize_doc(tokenized_doc: List[str], allowed_postags:list=['NOUN', 'ADJ', 'VERB', 'ADV']) -> List[str]:
    '''Document lemmatization using SpaCy
        
        Ref:
        - https://spacy.io/api/annotation
    '''
    
    spacy_doc = nlp(" ".join(tokenized_doc))
    doc_out = []
    for token in spacy_doc:
        if token.pos_ in allowed_postags:
            doc_out.append(token.lemma_)
        else:
            doc_out.append(token.text)
    return doc_out

<IPython.core.display.Javascript object>

In [14]:
def preprocess_and_tokenize_corpus(corpus: List[str],
                      stopwords: List[str],
                      allowed_postags:list=['NOUN', 'ADJ', 'VERB', 'ADV']) -> List[List[str]]:
    '''
    Preprocesses a corpus of texts. 
    '''
    
    preprocessed_corpus = []
    for doc in corpus:
        if len(" ".join(doc))==0 or len(" ".join(doc))>1000000:
            pass
        else:
            # Remove Figure No.'s
            doc = re.sub('Figure\s+\d+', '', doc)
            # Remove all numbers
            doc = re.sub('\d+', '', doc)
            # Remove new line characters
            doc = re.sub('\s+', ' ', doc)
            # Remove distracting single quotes
            doc = re.sub("\'", '', doc)
            # Run Gensim's simple_preprocesser - this tokenizes the document
            doc = simple_preprocess(str(doc), deacc=True)
            # Remove stopwords
            doc = [word for word in doc if word not in stopwords]
            # Lemmatize
            doc = lemmatize_doc(doc)
            
            preprocessed_corpus.append(doc)
    return preprocessed_corpus

<IPython.core.display.Javascript object>

In [15]:
# Pre-process and tokenize corpus
tokenized_corpus = preprocess_and_tokenize_corpus(corpus=corpus, stopwords=stop_words)

<IPython.core.display.Javascript object>

In [16]:
# View part of pre-processed corpus
pprint(tokenized_corpus[:1])

[['combine',
  'reporting',
  'application',
  'pende',
  'status',
  'sheet',
  'kalgoorlie',
  'sh',
  'sheet',
  'kalgoorlie',
  'project',
  'operator',
  'cazaly',
  'resources',
  'limited',
  'author',
  'watts',
  'date',
  'september',
  'distribution',
  'department',
  'industry',
  'resource',
  'cazaly',
  'resources',
  'limited',
  'cazaly',
  'resource',
  'limit',
  'september',
  'castle',
  'hill',
  'project',
  'tenement',
  'location',
  'plan',
  'scale',
  'regional',
  'geology',
  'exploration',
  'index',
  'map',
  'scale',
  'cazaly',
  'resource',
  'limit',
  'september',
  'summary',
  'combine',
  'mineral',
  'exploration',
  'report',
  'castle',
  'hill',
  'project',
  'detail',
  'exploration',
  'activity',
  'undertake',
  'cazaly',
  'resources',
  'limited',
  'reporting',
  'period',
  'july',
  'july',
  'report',
  'group',
  'know',
  'castle',
  'hill',
  'comprise',
  'nine',
  'grant',
  'prospect',
  'licence',
  'project',
  'area',
  

<IPython.core.display.Javascript object>

### Creating bigram and trigrams - Gensim’s Phrases model

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In this notebook, we focus on building a bigram model. However the code that is required to build a trigram model is made available (it just needs to be converted to a code cell).

In [17]:
bigram = gensim.models.Phrases(tokenized_corpus, min_count=5, threshold=100) # higher threshold fewer phrases.

<IPython.core.display.Javascript object>

Faster way to get a sentence clubbed as a trigram/bigram

In [18]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

<IPython.core.display.Javascript object>

View example of bigram model

In [19]:
# See example of bigram model applied to pre-processed corpus
pprint(bigram_mod[tokenized_corpus[0]])

['combine',
 'reporting',
 'application',
 'pende',
 'status',
 'sheet',
 'kalgoorlie',
 'sh',
 'sheet',
 'kalgoorlie',
 'project',
 'operator',
 'cazaly_resources',
 'limited',
 'author',
 'watts',
 'date',
 'september',
 'distribution',
 'department_industry',
 'resource',
 'cazaly_resources',
 'limited',
 'cazaly',
 'resource',
 'limit',
 'september',
 'castle_hill',
 'project',
 'tenement',
 'location',
 'plan',
 'scale',
 'regional',
 'geology',
 'exploration',
 'index',
 'map',
 'scale',
 'cazaly',
 'resource',
 'limit',
 'september',
 'summary',
 'combine',
 'mineral',
 'exploration',
 'report',
 'castle_hill',
 'project',
 'detail',
 'exploration',
 'activity',
 'undertake',
 'cazaly_resources',
 'limited',
 'reporting_period',
 'july',
 'july',
 'report',
 'group',
 'know',
 'castle_hill',
 'comprise',
 'nine',
 'grant',
 'prospect',
 'licence',
 'project',
 'area',
 'cover',
 'hectare_cazaly',
 'register_holder',
 'castle_hill',
 'project',
 'tenement',
 'cazalys',
 'explorat

<IPython.core.display.Javascript object>

Convert tokenized corpus into bi-gram phrases - this will allow the topic model to capture domain specific phrases in addition to individual words

In [20]:
bigram_corpus = [bigram_mod[tokenized_doc] for tokenized_doc in tokenized_corpus]

<IPython.core.display.Javascript object>

To perform topic modelling with LDA we need to convert the corpus of pre-processed texts into a bag-of-words (bow) representation with count frequencies.

In [21]:
# Create a dictionary that maps each word to a unique id
id2word = corpora.Dictionary(bigram_corpus)

<IPython.core.display.Javascript object>

In [22]:
id2word[162]

'project'

<IPython.core.display.Javascript object>

Convert the bigram corpus into a [bag-of-words (bow) representation](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

In [23]:
bow_corpus = [id2word.doc2bow(text) for text in bigram_corpus]

<IPython.core.display.Javascript object>

In [24]:
# Review a single document in bow format; (index, count frequency) 
print(bow_corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 8), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 17), (13, 3), (14, 1), (15, 1), (16, 4), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 13), (26, 8), (27, 5), (28, 3), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 4), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 4), (47, 2), (48, 1), (49, 1), (50, 1), (51, 2), (52, 3), (53, 2), (54, 2), (55, 1), (56, 1), (57, 3), (58, 1), (59, 3), (60, 1), (61, 1), (62, 4), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 13), (70, 1), (71, 1), (72, 1), (73, 3), (74, 1), (75, 2), (76, 2), (77, 2), (78, 3), (79, 1), (80, 1), (81, 1), (82, 5), (83, 4), (84, 3), (85, 3), (86, 1), (87, 4), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 1), (94, 1), (95, 2), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 4), (106, 7), (107, 1), (108, 1), (109, 1), (110,

<IPython.core.display.Javascript object>

In [25]:
# View the corresponding corpus with bigrams
print(bigram_corpus[0])

['combine', 'reporting', 'application', 'pende', 'status', 'sheet', 'kalgoorlie', 'sh', 'sheet', 'kalgoorlie', 'project', 'operator', 'cazaly_resources', 'limited', 'author', 'watts', 'date', 'september', 'distribution', 'department_industry', 'resource', 'cazaly_resources', 'limited', 'cazaly', 'resource', 'limit', 'september', 'castle_hill', 'project', 'tenement', 'location', 'plan', 'scale', 'regional', 'geology', 'exploration', 'index', 'map', 'scale', 'cazaly', 'resource', 'limit', 'september', 'summary', 'combine', 'mineral', 'exploration', 'report', 'castle_hill', 'project', 'detail', 'exploration', 'activity', 'undertake', 'cazaly_resources', 'limited', 'reporting_period', 'july', 'july', 'report', 'group', 'know', 'castle_hill', 'comprise', 'nine', 'grant', 'prospect', 'licence', 'project', 'area', 'cover', 'hectare_cazaly', 'register_holder', 'castle_hill', 'project', 'tenement', 'cazalys', 'exploration', 'activity', 'report', 'period', 'direct_towards', 'acquisition', 'avail

<IPython.core.display.Javascript object>

In [26]:
# Sort documents by term frequency e.g. most frequency first
sorted_by_value = sorted(bow_corpus[0], key=lambda kv: kv[1], reverse=True)
print(sorted_by_value)

[(162, 27), (12, 17), (25, 13), (69, 13), (173, 10), (207, 10), (3, 8), (26, 8), (106, 7), (177, 7), (136, 6), (214, 6), (227, 6), (27, 5), (82, 5), (116, 5), (124, 5), (159, 5), (182, 5), (16, 4), (35, 4), (46, 4), (62, 4), (83, 4), (87, 4), (105, 4), (112, 4), (115, 4), (117, 4), (170, 4), (174, 4), (176, 4), (209, 4), (1, 3), (13, 3), (28, 3), (52, 3), (57, 3), (59, 3), (73, 3), (78, 3), (84, 3), (85, 3), (129, 3), (134, 3), (140, 3), (151, 3), (154, 3), (160, 3), (166, 3), (172, 3), (178, 3), (189, 3), (194, 3), (213, 3), (233, 3), (5, 2), (6, 2), (10, 2), (29, 2), (37, 2), (42, 2), (44, 2), (47, 2), (51, 2), (53, 2), (54, 2), (66, 2), (75, 2), (76, 2), (77, 2), (92, 2), (95, 2), (110, 2), (114, 2), (123, 2), (125, 2), (127, 2), (128, 2), (137, 2), (150, 2), (158, 2), (163, 2), (171, 2), (175, 2), (179, 2), (183, 2), (187, 2), (188, 2), (191, 2), (195, 2), (200, 2), (203, 2), (216, 2), (217, 2), (0, 1), (2, 1), (4, 1), (7, 1), (8, 1), (9, 1), (11, 1), (14, 1), (15, 1), (17, 1), (18

<IPython.core.display.Javascript object>

In [27]:
# View a human readable format of corpus (term-frequency)
[(id2word[id], freq) for id, freq in sorted_by_value] 

[('project', 27),
 ('area', 17),
 ('castle_hill', 13),
 ('exploration', 13),
 ('report', 10),
 ('tenement', 10),
 ('activity', 8),
 ('cazaly', 8),
 ('kalgoorlie', 7),
 ('review', 7),
 ('nickel', 6),
 ('ultramafic', 6),
 ('within', 6),
 ('cazaly_resources', 5),
 ('geology', 5),
 ('limit', 5),
 ('mafic', 5),
 ('potential', 5),
 ('september', 5),
 ('available', 4),
 ('combine', 4),
 ('coolgardie', 4),
 ('domain', 4),
 ('gold', 4),
 ('group', 4),
 ('july', 4),
 ('kunanalle', 4),
 ('lie', 4),
 ('limited', 4),
 ('regional', 4),
 ('reporting', 4),
 ('resource', 4),
 ('terrane', 4),
 ('acquisition', 3),
 ('assess', 3),
 ('cazalys', 3),
 ('dataset', 3),
 ('detail', 3),
 ('direct_towards', 3),
 ('fault', 3),
 ('follow', 3),
 ('grant', 3),
 ('greenstone', 3),
 ('mineral', 3),
 ('nearby', 3),
 ('north', 3),
 ('part', 3),
 ('period', 3),
 ('previous', 3),
 ('rate', 3),
 ('relate', 3),
 ('rock', 3),
 ('sheet', 3),
 ('structural', 3),
 ('trend', 3),
 ('zone', 3),
 ('adjacent', 2),
 ('along', 2),
 ('a

<IPython.core.display.Javascript object>

## Train LDA Model <a name="training"></a>

<img src="https://miro.medium.com/max/800/1*pZo_IcxW1GVuH2vQKdoIMQ.jpeg" alt="LDA example" style="width:600px;"/>

Here we train a LDA topic mode on our bag-of-words corpus. We initially set the `number of topics` we believe are in the corpus to 10. Only a single `pass` is performed through the corpus, but this could be increased to get better results. However, it is computationally expensive.

In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1,   # To get an improved LDA model, this can be increased
                                           alpha='auto',
                                           per_word_topics=True)

<IPython.core.display.Javascript object>

Here we show the topics elicited from the LDA model and the top keywords in the 10 topics. This shows the weighted importance of each word in each topic.

In [29]:
pprint(lda_model.print_topics())

[(0,
  '0.015*"area" + 0.015*"tenement" + 0.014*"gold" + 0.012*"project" + '
  '0.012*"exploration" + 0.010*"report" + 0.008*"within" + 0.008*"rock" + '
  '0.008*"australia" + 0.007*"western"'),
 (1,
  '0.015*"gold" + 0.013*"area" + 0.011*"sample" + 0.010*"exploration" + '
  '0.010*"tenement" + 0.010*"project" + 0.009*"report" + 0.007*"rock" + '
  '0.007*"drilling" + 0.006*"soil"'),
 (2,
  '0.008*"quartz" + 0.007*"vein" + 0.007*"carbonate" + 0.007*"fine" + '
  '0.006*"deposit" + 0.006*"pyrite" + 0.006*"sf" + 0.005*"iron" + '
  '0.005*"matrix" + 0.004*"mineralisation"'),
 (3,
  '0.013*"resource" + 0.012*"gold" + 0.011*"grade" + 0.011*"hole" + '
  '0.009*"zone" + 0.009*"drill" + 0.009*"pit" + 0.008*"drilling" + '
  '0.008*"sample" + 0.007*"mineralisation"'),
 (4,
  '0.019*"datum" + 0.016*"survey" + 0.011*"area" + 0.008*"use" + '
  '0.008*"magnetic" + 0.007*"gravity" + 0.006*"report" + 0.006*"base" + '
  '0.006*"line" + 0.006*"sample"'),
 (5,
  '0.018*"sample" + 0.013*"hole" + 0.012*"surv

<IPython.core.display.Javascript object>

In [30]:
doc_lda = lda_model[bow_corpus]

<IPython.core.display.Javascript object>

To evaluate the quality of our topic model, we can compute its `perplexity` and `coherence`.

In [31]:
# Compute Perplexity - lower is better.
print(f'Perplexity: {lda_model.log_perplexity(bow_corpus)}')

Perplexity: -8.497820323965014


<IPython.core.display.Javascript object>

In [32]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=bigram_corpus, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.45187832443979614


<IPython.core.display.Javascript object>

## Interactive LDA Visualisation <a name="visualisation"></a> 

#### Wait, what am I looking at again?

There are a lot of moving parts in the visualization. Here's a brief summary:

On the left, there is a plot of the "distance" between all of the topics (labeled as the Intertopic Distance Map)
The plot is rendered in two dimensions according a multidimensional scaling (MDS) algorithm. Topics that are generally similar should be appear close together on the plot, while dissimilar topics should appear far apart.

The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.

An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.

On the right, there is a bar chart showing top terms.
When no topic is selected in the plot on the left, the bar chart shows the top most "salient" terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.

When a particular topic is selected, the bar chart changes to show the top most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter λλ, which can be adjusted with a slider above the bar chart.

Setting the λλ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.

Setting λλ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic — i.e., terms that occur only in this topic, and do not occur in other topics.

Setting λλ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.

Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

A more detailed explanation of the pyLDAvis visualization can be found here. Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's LdaMulticore object and pyLDAvis' visualization, you have to dig through the terms manually.

#### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:

Better understanding and interpreting individual topics, and Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the λλ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

#### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% Topic A, 20% Topic B, 20% Topic C, and 10% Topic D.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:

Using spaCy to remove punctuation and lemmatize the text

- Applying our first-order phrase model to join word pairs
- Applying our second-order phrase model to join longer phrases
- Removing stopwords
- Creating a bag-of-words representation
- Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The lda_description(...) function will perform all these steps for us, including printing the resulting topical description of the input text.

##### Acknowledgement: https://www.kaggle.com/navinch/interesting-visualizations-lda-word2vec

### Visualize the topics

In [33]:
pyLDAvis.enable_notebook()

<IPython.core.display.Javascript object>

In [34]:
vis = gensimvis.prepare(lda_model, bow_corpus, id2word)  # version 3.3+ use pyLDAvis.gensim_models.
vis

c:\Users\Tyler\AppData\Local\Programs\Python\Python38\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.097343  0.013475       1        1  22.383342
3      0.046600  0.003946       2        1  18.346886
1      0.070536  0.012837       3        1  16.429138
5      0.078696 -0.002911       4        1  13.176983
4      0.020639  0.062944       5        1   8.315149
9      0.072538 -0.180609       6        1   8.175331
6     -0.005534  0.122461       7        1   6.869653
7     -0.042780  0.090077       8        1   4.269823
8     -0.228634 -0.006625       9        1   1.051949
2     -0.109405 -0.115594      10        1   0.981745, topic_info=        Term          Freq         Total Category  logprob  loglift
1084  survey   9407.000000   9407.000000  Default  30.0000  30.0000
12      area  16526.000000  16526.000000  Default  29.0000  29.0000
915   quartz   5167.000000   5167.000000  Default  28.0000  28.0000
54     datum   7935.000000   7935.000000  Default  27.0000  27.0000
1166    vein   2933.000000   2933.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
178     rock     58.030123  10496.572174  Topic10  -5.5924  -0.5742
975   sample     57.154921  14564.535444  Topic10  -5.6076  -0.9170
3108    rich     46.012700   1467.023465  Topic10  -5.8245   1.1615
12      area     51.860010  16526.752644  Topic10  -5.7048  -1.1406
233     zone     47.975207   8010.714718  Topic10  -5.7827  -0.4943

[856 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
5004      1  0.079886  aboriginal
5004      2  0.002219  aboriginal
5004      3  0.084324  aboriginal
5004      4  0.019971  aboriginal
5004      5  0.017752  aboriginal
...     ...       ...         ...
233       6  0.054177        zone
233       7  0.009362        zone
233       8  0.016603        zone
233       9  0.002122        zone
233      10  0.005992        zone

[3645 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 6, 5, 10, 7, 8, 9, 3])

<IPython.core.display.Javascript object>

## Additional Content: Tuning the number of topics (hyperparameter tuning) <a name="tuning"></a>

Here we explore how to find the best number of topics for our LDA model. This involves iteratively computing coherence values and finding an optimal topic distribution over a given number of topics.

⚠️ Computing coherence values can take a very long time to run.

Here we can visualise coherence scores as a function of the number of topics